# Testing for Differences in Distributions via Mean and Variance Comparisons

In this notebook, I explore the comparison of two distributions from a control group and a treatment group. The treatment group has a much lower variance when compared to the control group, leading some in the lab to think the treatment is a success. However, Dr. Cudmore is skpetical, and believes the lowered variance is due to measurement error and/or lost of signal. Thus we search to find the following: is the variance difference of a linear shift or a multiplicative scale? If so, can we produce a confidence interval to for the true transformation parameter?

In [1]:
# Auto formatting
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
import scipy.stats as sp
import plotly.express as px
from IPython.display import display
from scipy.stats import bootstrap
import seaborn as sns

ImportError: cannot import name 'bootstrap' from 'scipy.stats' (C:\Users\Gianni\anaconda3\lib\site-packages\scipy\stats\__init__.py)

<IPython.core.display.Javascript object>

## Simulated Data

In [ ]:
# We generate random data, from different normal distributions
rng = default_rng()
dist1 = rng.normal(5, 3, 1000)
dist2 = rng.normal(15, 3, 1000)

A two sample Kolmogorov-Smirnov test is a non-parametetric method of comparing two samples by testing if they come from the same (but unknown) distribution. Our null hypothesis being $H_0: $ "There is no difference between the two distributions." 

In [ ]:
# Kolmogorov Smirnoff Test
sp.kstest(dist1, dist2)

This extremely small p-value should not be surprising, as we know these samples are from two very different normal distributions.

The next idea would be to do a bootstrap for the difference and proportions in both mean and variance, as to see if we can approxzimate the difference between these distributions via constant resampling. Below is my newer, easier way of performing the bootstrap.

Here, I perform the bootstrap without any packages, however I only perform it for the variance. The same can easily be done for the mean. 

In [ ]:
# Approaching comparing variances with bootstrap approach
# np.var(dist1) - np.var(dist2)

var_sub = np.array([])
var_div = np.array([])
for i in range(1000):
    samp1 = rng.choice(dist1, size=1000, replace=True)
    samp2 = rng.choice(dist2, size=1000, replace=True)

    # print(np.var(samp1))
    # We get both the difference in variances (theta1) and the
    theta1 = np.var(samp1) - np.var(samp2)  # if same, should be centered around 0
    theta2 = np.var(samp1) / np.var(samp2)  # if same, should be centered around 1

    # Append both numpy array
    var_sub = np.append(var_sub, theta1)
    var_div = np.append(var_div, theta2)


fig1 = px.histogram(var_sub, title="Differences in Variances")
fig1.show()
print(np.mean(var_sub))

fig2 = px.histogram(var_div, title="Proportionality of Variances")
fig2.show()

print(np.mean(var_div))

When we have two distributions, we can compare for the difference in variances as well as the proportionality of the variances with a bootstrap. This can also allow us to create confidence intervals (multiple types, ideally the BCa interval). If these variances are the same, we would expect that within a confidence interval for the differences we would see zero and with a proportionality of one. If these values are not contained in a confindence interval, we could then find how these variances differ. 

Fortunately, scipy recently came out with a bootstrap function that produces confidence intervals. However, as of writing, the BCa confidence interval can only be done for a one sampled bootstrap. However, the 'basic' bootstrap method in Scipy also corrects for skewness and supports two samples, so we can proceed with confidence that we will not be getting an over estimate, like the percentile CI. 

In [ ]:
# testing scipy bootstrap
# This is incorrect 

# sample4boot = dist1 / dist2
# sample4boot = (sample4boot,)
# res = bootstrap(
#     sample4boot,
#     np.var,
#     vectorized=False,
#     n_resamples=1000,
#     method="BCa",
#     random_state=rng,
# )
# res.confidence_interval

Before we start using the real data, we start off with some simply functions that will deliver us the desired results of our bootstrap that we can provide into the spicy bootstrap function. 

In [ ]:
# Testing Scipy bootstrap
# run this if not installing to update scipy
# !pip install -U scipy
def meandiff(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)


def meanscale(sample1, sample2):
    return np.mean(sample1) / np.mean(sample2)


def vardiff(sample1, sample2):
    return np.var(sample1) - np.var(sample2)


def varscale(sample1, sample2):
    return np.var(sample1) / np.var(sample2)


# Using Real Data

Moving on from simulation, now I will compare the means and variances with bootstrap using some real data from the lab. First I will copy paste what was provided by Dr. Cudmore, with the addition of reading the csv files from urls instead of file paths on a system (just a preference). 

In [3]:
# we can add to this as we get more data
pathList = []

# I changed this to urls so that it would be accesable from any device
# pathList.append("dataGianni/masterDb-jan-12-2022.csv")
# pathList.append("dataGianni/masterDb-jan-18-2022.csv")
# pathList.append("dataGianni/masterDb-feb-15-2022.csv")
pathList.append(
    "https://raw.githubusercontent.com/gspiga/Cudmore/main/VarAnalysis/gianni_var_analysis/dataGianni/masterDb-jan-12-2022.csv"
)
pathList.append(
    "https://raw.githubusercontent.com/gspiga/Cudmore/main/VarAnalysis/gianni_var_analysis/dataGianni/masterDb-jan-18-2022.csv"
)
pathList.append(
    "https://raw.githubusercontent.com/gspiga/Cudmore/main/VarAnalysis/gianni_var_analysis/dataGianni/masterDb-feb-15-2022.csv"
)


# make a list of dataframe (from csv files)
dfList = []
for fileIdx, path in enumerate(pathList):
    dfPath = pd.read_csv(path)
    dfPath["myFileIdx"] = fileIdx  # add for our book keeping if necc
    dfList.append(dfPath)

# make a single dataframe from all files in list
dfMaster = pd.concat(dfList, ignore_index=True)
display(dfMaster)

,Unnamed: 0,analysisVersion,interfaceVersion,file,detectionType,cellType,sex,condition,sweep,sweepSpikeNumber,...,diastolicDuration_ms,widths,widths_10,widths_20,widths_50,widths_80,widths_90,myDateStr,fileIdx,myFileIdx
0,0,20210803a,20210803a,220110n_0003.tif,mv,NaN,NaN,Control,0,0,...,23.5704,"[{'halfHeight': 10, 'risingPnt': 96, 'fallingP...",769.9664,742.4676,557.8328,396.7684,302.4868,jan-12-2022,0,0
1,1,20210803a,20210803a,220110n_0003.tif,mv,NaN,NaN,Control,0,1,...,3.9284,"[{'halfHeight': 10, 'risingPnt': 347, 'falling...",785.6800,754.2528,589.2600,428.1956,345.6992,jan-12-2022,0,0
2,2,20210803a,20210803a,220110n_0003.tif,mv,NaN,NaN,Control,0,2,...,3.9284,"[{'halfHeight': 10, 'risingPnt': 598, 'falling...",809.2504,754.2528,581.4032,408.5536,282.8448,jan-12-2022,0,0
3,3,20210803a,20210803a,220110n_0003.tif,mv,NaN,NaN,Control,0,3,...,19.6420,"[{'halfHeight': 10, 'risingPnt': 849, 'falling...",895.6752,726.7540,581.4032,396.7684,259.2744,jan-12-2022,0,0
4,4,20210803a,20210803a,220110n_0003.tif,mv,NaN,NaN,Control,0,4,...,3.9284,"[{'halfHeight': 10, 'risingPnt': 1100, 'fallin...",813.1788,714.9688,601.0452,381.0548,243.5608,jan-12-2022,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,65,20210803a,20210803a,thapsi 2.5_0047.tif,mv,NaN,NaN,TG,0,0,...,51.5350,"[{'halfHeight': 10, 'risingPnt': 169, 'falling...",688.6950,627.7900,505.9800,187.4000,182.7150,feb-15-2022,8,2
396,66,20210803a,20210803a,thapsi 2.5_0047.tif,mv,NaN,NaN,TG,0,1,...,18.7400,"[{'halfHeight': 10, 'risingPnt': 346, 'falling...",773.0250,726.1750,580.9400,281.1000,79.6450,feb-15-2022,8,2
397,67,20210803a,20210803a,thapsi 2.5_0047.tif,mv,NaN,NaN,TG,0,2,...,79.6450,"[{'halfHeight': 10, 'risingPnt': 541, 'falling...",754.2850,716.8050,557.5150,248.3050,182.7150,feb-15-2022,8,2
398,68,20210803a,20210803a,thapsi 2.5_0047.tif,mv,NaN,NaN,TG,0,3,...,28.1100,"[{'halfHeight': 10, 'risingPnt': 720, 'falling...",482.5550,454.4450,351.3750,154.6050,84.3300,feb-15-2022,8,2


<IPython.core.display.Javascript object>

In [ ]:
# From meeting on 3/7, just trying different histograms
# subset = dfMaster["peakVal"][dfMaster["file"] == "220110n_0014.tif"]
# figure = px.histogram(subset)
# figure.show()

# subset = dfMaster["peakVal"][dfMaster["file"] == "2.5Hz_ctrl_0012.tif"]
# figure = px.histogram(subset)
# figure.show()


In [ ]:
# these are all the columns we have, some are useful and some are not
print(dfMaster.columns)

## Useful columns

```
myFileIdx: index of csv we were loaded
file: name of the file
condition: condiiton of recording from ('Control', 'TP')
thresholdSec: time of threshold
thresholdVal: value at threshold
peakSec: time of peak
peakVal: value at peak  # look at the variance
peakHeight: height of peak (peakVal - thresholdVal)
isi_ms: time between peak[i] and peak[i-1]
```

## Clean up the data

Replace conditon 'TG' with 'Thapsigargin'

In [4]:
condList = dfMaster["condition"].unique()
print(f"before condList:{condList}")

# do not do this
# dfMaster[ dfMaster['condition']=='TG' ]['condition'] = 'Thapsigargin'

# do this
dfMaster.loc[dfMaster["condition"] == "TG", "condition"] = "Thapsigargin"

condList = dfMaster["condition"].unique()
print(f"after condList:{condList}")

before condList:['Control' 'Thapsigargin' 'TG']
after condList:['Control' 'Thapsigargin']


<IPython.core.display.Javascript object>

## Plot stats in a table

In [5]:
# could set this to any column to get stats
colStr = "peakVal"

# a list of stats to take per 'file'
# changed scipy.stats to sp
aggList = ["count", "min", "max", "mean", sp.sem, "median", "var"]

dfStats = dfMaster.groupby(["file", "condition", "myFileIdx"], as_index=False)[
    colStr
].agg(aggList)

print(f"stats for col: {colStr}")
display(dfStats)

stats for col: peakVal


,,,count,min,max,mean,sem,median,var
file,condition,myFileIdx,,,,,,,
2.5Hz_ctrl_0005.tif,Control,1,13,2.818897,3.118840,2.964127,0.027872,2.983678,0.010099
2.5Hz_ctrl_0009.tif,Control,1,8,2.607111,2.763000,2.690624,0.021312,2.689034,0.003634
2.5Hz_ctrl_0012.tif,Control,1,13,2.758010,3.176939,2.979535,0.040233,3.034222,0.021043
2.5Hz_ctrl_0017.tif,Control,1,9,1.787921,2.417780,2.304765,0.065323,2.362354,0.038404
2.5Hz_ctrl_0028.tif,Control,1,10,2.430220,2.687595,2.582455,0.026511,2.597892,0.007028
2.5Hz_thapsi_0024.tif,Thapsigargin,1,10,1.196586,1.292551,1.235665,0.009530,1.227982,0.000908
220110n_0003.tif,Control,0,10,1.912594,2.003728,1.957616,0.010250,1.965467,0.001051
220110n_0005.tif,Control,0,6,2.232398,2.412802,2.317046,0.028719,2.323407,0.004949
220110n_0009.tif,Control,0,5,2.588777,2.660828,2.621502,0.012971,2.618259,0.000841


<IPython.core.display.Javascript object>

This is where I will branch off a bit (I won't include the plots for sake of repetitvity). My first idea would be to split this data into two groups, by condition. 

In [ ]:
# Data frame with only control group, using method for multilevel index
dfCont = dfStats[dfStats.index.isin(["Control"], level=1)]

# Data frame for Thapsigargin
dfThap = dfStats[dfStats.index.isin(["Thapsigargin"], level=1)]

display(dfCont.head())
display(dfThap.head())

The big question: "Is the low variance (var) in 'Thapsigargin' condition simply due to the fact that the amplitude of the peak (mean) is small?" 

In [ ]:
# hold off on using style='file', legend is too long and needs to be tweaked
# sns.scatterplot(x='mean', y='var', hue='condition', style='file', data=dfStats)

# Seaborn
sns.scatterplot(x="mean", y="var", hue="condition", style="myFileIdx", data=dfStats)

# Plotly
scat = px.scatter(
    dfStats,
    x="mean",
    y="var",
    color=dfStats.index.get_level_values(1),
    symbol=dfStats.index.get_level_values(2),
)
scat.show()

First, I'd like to see how these variables are distributed via histogram. Second, I would like how the distributions of both the mean and the variance for both groups compare to each other. To do this, we perform a Two-Sample Kolmogrov-Smirnov test. Where we test, "What is the probability that these two sets of samples were drawn from the same (but unknown) probability distribution?" Recall that $H_0$ is that these samples are drawn from the same distribution. 

In [ ]:
# Lets look at what the distributions of this data actually looks like
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Subplots for the Mean
subplot_mean = make_subplots(rows=1, cols=2, shared_yaxes=True)
subplot_mean.add_trace(
    go.Histogram(x=dfCont["mean"], histnorm="probability", nbinsx=15), row=1, col=1
)
subplot_mean.add_trace(
    go.Histogram(x=dfThap["mean"], histnorm="probability", nbinsx=15,), row=1, col=2
)

subplot_mean.update_layout(
    height=600,
    width=800,
    title_text="Distribution of Means for Control and Thapsigargin",
)
subplot_mean.show()

# Now plotting the variance subplot
subplot_var = make_subplots(rows=1, cols=2, shared_yaxes=True)

subplot_var.add_trace(
    go.Histogram(x=dfCont["var"], nbinsx=15, histnorm="probability",), row=1, col=1
)
subplot_var.add_trace(
    go.Histogram(x=dfThap["var"], nbinsx=15, histnorm="probability",), row=1, col=2
)

subplot_var.update_layout(
    height=600,
    width=800,
    title_text="Distribution of Variances for Control and Thapsigargin",
)
subplot_var.show()

#Cumulative Histogram comparison

## Bootstrapping

In [ ]:
# KS test for the mean
display(sp.kstest(dfCont["mean"], dfThap["mean"]))

# ... and for the variance
display(sp.kstest(dfCont["var"], dfThap["var"]))


These results would leave us to reject $H_0$ for both statistics with any reasonable level of $\alpha$, leaving us to conclude that both the mean and variance of these groups do not come from the same distribution. Now we are interested in bootstrapping for both mean and variance differences, both subtrating and multiplying.

In [ ]:
rng = default_rng(seed=12345)

# For scipy bootstrap
mean_data = (dfCont["mean"], dfThap["mean"])

mean_sub = np.array([])
mean_div = np.array([])
for i in range(1000):
    samp1 = rng.choice(dfCont["mean"], size=len(dfCont["mean"]), replace=True)
    samp2 = rng.choice(dfThap["mean"], size=len(dfThap["mean"]), replace=True)

    # print(np.var(samp1))
    # We get both the difference in variances (theta1) and the
    theta1 = np.mean(samp1) - np.mean(samp2)  # if same, should be centered around 0
    theta2 = np.mean(samp1) / np.mean(samp2)  # if same, should be centered around 1

    # Append both numpy array
    mean_sub = np.append(mean_sub, theta1)
    mean_div = np.append(mean_div, theta2)


fig1 = px.histogram(
    mean_sub,
    histnorm="probability",
    title="Differences of Means between Control and Thapsigargin",
)
fig1.show()

# Bootstrap ci for difference in means
mean_diff_bs = bootstrap(
    mean_data,
    meandiff,
    n_resamples=1000,
    method="basic",
    vectorized=False,
    random_state=rng,
)
mean_diff_ci = mean_diff_bs.confidence_interval
print(mean_diff_ci)
print(
    "The average mean difference between control and the Thapsigargin mean is:",
    np.mean(mean_sub),
)
fig2 = px.histogram(
    mean_div,
    histnorm="probability",
    title="Scale of Means between Control and Thapsigargin",
)
fig2.show()

# Bootstrap ci for scale in means
mean_scale_bs = bootstrap(
    mean_data,
    meanscale,
    n_resamples=1000,
    method="basic",
    vectorized=False,
    random_state=rng,
)
mean_scale_ci = mean_scale_bs.confidence_interval
print(mean_scale_ci)

print(
    "The average mean scale between the control and the Thapsigargin mean is:",
    np.mean(mean_div),
)

In [ ]:
rng = default_rng(seed=12345)

var_data = (dfCont["var"], dfThap["var"])
var_sub = np.array([])
var_div = np.array([])


for i in range(1000):
    samp1 = rng.choice(dfCont["var"], size=len(dfCont["var"]), replace=True)
    samp2 = rng.choice(dfThap["var"], size=len(dfThap["var"]), replace=True)

    # print(np.var(samp1))
    # We get both the difference in variances (theta1) and the
    theta1 = np.mean(samp1) - np.mean(samp2)  # if same, should be centered around 0
    theta2 = np.mean(samp1) / np.mean(samp2)  # if same, should be centered around 1

    # Append both numpy array
    var_sub = np.append(var_sub, theta1)
    var_div = np.append(var_div, theta2)


fig1 = px.histogram(
    var_sub,
    histnorm="probability",
    title="Differences of Variances between Control and Thapsigargin",
)
fig1.show()

# Bootstrap ci for difference in variance
var_diff_bs = bootstrap(
    var_data,
    meandiff,
    n_resamples=1000,
    method="basic",
    vectorized=False,
    random_state=rng,
)
var_diff_ci = var_diff_bs.confidence_interval
print(var_diff_ci)
print(
    "The average mean difference between the control and the Thapsigargin variance is:",
    np.mean(var_sub),
)

fig2 = px.histogram(
    var_div,
    histnorm="probability",
    title="Scale of Variances between Control and Thapsigargin",
)
fig2.show()

var_scale_bs = bootstrap(
    var_data,
    meanscale,
    n_resamples=1000,
    method="basic",
    vectorized=False,
    random_state=rng,
)
var_scale_ci = var_scale_bs.confidence_interval
print(var_scale_ci)
print(
    "The average mean scale between the control and the Thapsigargin variance is:",
    np.mean(var_div),
)

## Concluding our Bootstrap

The key detail to pay attention to in our bootstrap histograms is the scale of variances. We know by laws of variance:
$$
Var(X + c) = E[((X + c) - E(X + c))^2] = Var(X)\\
Var(cX) = c^2Var(X)
$$
Where $c \in \mathbb{R}$. If this data were to be solely a linear shift, the means would change linearly (a difference $\neq$ 0), which we observe. However we would expect the difference of variances to be centered around 0 since in a linear shift they should be, in essence, the same. However since we do not see zero within our 95% confidence interval, we can conclude that the transformation from the control group to the Thasigargin group is a multiplicative shift. 

#### Some Resources Referenced

https://medium.com/@wenjun.sarah.sun/bootstrap-confidence-interval-in-python-3fe8d5a6fd56

https://bashtage.github.io/arch/bootstrap/confidence-intervals.html

http://bebi103.caltech.edu.s3-website-us-east-1.amazonaws.com/2019a/content/recitations/bootstrapping.html

###  Checking Correlations

In [ ]:
### Let's check some correlations here
# Is the low variance (var) in 'Thapsigargin' condition simply due to the fact that the amplitude of the peak (mean) is small?

#What is the relationship between the variance and mean of the control group?
print(np.corrcoef(dfCont["var"], dfCont["mean"])[0, 1])

#What is the relationship between the variance and mean of the Thpsigargin group
print(np.corrcoef(dfThap["var"], dfThap["mean"])[0, 1])


# Predicting Peak with Variance Behavior

In [ ]:
dfUseful = dfMaster[
    [
        "myFileIdx",
        "file",
        "condition",
        "thresholdSec",
        "thresholdVal",
        "peakSec",
        "peakVal",
        "peakHeight",
        "isi_ms",
        "spikeNumber",
    ]
]
# dfUseful.groupby(["file", "condition", "myFileIdx"], as_index=False)
# dfUseful.set_index(["file", "condition", "myFileIdx"], inplace=True)
display(dfUseful)

In [ ]:
# recreate line plot with plotly
line_fig = px.line(dfUseful, x="spikeNumber", y="peakVal", color="condition")
line_fig.show()

# This might be harder than I thought

# import plotly.graph_objects as go

# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=dfUseful["spikeNumber"], y=dfUseful["peakVal"], color=dfUseful["condition"]
#     )
# )

# fig.show()